# Convert t3r files to Photon-HDF5

# 2. Prepare the data file

Before starting, you have to upload a data file to be converted to Photon-HDF5.
You can use one of our example data files available
[on figshare](https://figshare.com/articles/data_files_for_phconvert/5421565). 


Specify the input data file in the following cell:

In [ ]:
filename = 'data/Point_11_s23d9A15_70%25sucrose_55%25.t3r'

In [ ]:
import os
try: 
    with open(filename): pass
    print('Data file found, you can proceed.')
except IOError:
    print('ATTENTION: Data file not found, please check the filename.\n'
          '           (current value "%s")' % filename)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
import numpy as np
import phconvert as phc
print('phconvert version: ' + phc.__version__)

In [ ]:
d, meta = phc.loader.nsalex_t3r(filename,
                                donor = 1,
                                acceptor = 0,
                                alex_period_donor = (3000, 4000),
                                alex_period_acceptor = (0, 2000),
                                excitation_wavelengths = (470e-9, 635e-9),
                                detection_wavelengths = (525e-9, 690e-9),
                                )

In [ ]:
phc.plotter.alternation_hist(d)

In [ ]:
detectors = d['photon_data']['detectors']

print("Detector    Counts")
print("--------   --------")
for det, count in zip(*np.unique(detectors, return_counts=True)):
    print("%8d   %8d" % (det, count))

## Meta data

In [ ]:
author = 'Biswajit'
author_affiliation = 'Leiden University'
description = 'A demonstrative pt3 data readin.'
sample_name = 'Fluorescence enhancement experiment with a gold nanorod'
dye_names = 'Alexa 647'
buffer_name = '70% sucrose solution'

### Add meta data

In [ ]:
d['description'] = description

d['sample'] = dict(
    sample_name=sample_name,
    dye_names=dye_names,
    buffer_name=buffer_name,
    num_dyes = len(dye_names.split(',')))

d['identity'] = dict(
    author=author,
    author_affiliation=author_affiliation)

In [ ]:
# Remove some empty groups that may cause errors on saving
_ = meta.pop('dispcurve', None)
_ = meta.pop('imghdr', None)

In [ ]:
d['user'] = {'picoquant': meta}

## Save to Photon-HDF5

In [ ]:
phc.hdf5.save_photon_hdf5(d, overwrite=True)

In [ ]:
# del d

## Load Photon-HDF5

In [ ]:
from pprint import pprint

In [ ]:
filename = d['_data_file'].filename

In [ ]:
h5data = phc.hdf5.load_photon_hdf5(filename)

In [ ]:
phc.hdf5.dict_from_group(h5data.identity)

In [ ]:
phc.hdf5.dict_from_group(h5data.setup)

In [ ]:
pprint(phc.hdf5.dict_from_group(h5data.photon_data))

In [ ]:
h5data._v_file.close()